# IMPORTS

In [ ]:
#import necessary dependecies
import os
import warnings
import numpy as np  
import pandas as pd, os, gc , random
from tqdm import tqdm_notebook as tqdm 
warnings.filterwarnings('ignore') 
np.random.seed(111)
random.seed(111)
from sklearn.model_selection import  StratifiedKFold

In [ ]:
import datetime
import pandas as pd
from time import time
# from autograd import grad
# import autograd.numpy as np
import numpy as np
from numba import njit
from scipy.optimize import minimize, fsolve
from sklearn.metrics import log_loss

# Get Data

In [ ]:
Train = pd.read_csv('Train.csv')
Train = Train[~Train.Image_id.str.contains('_rgn')].reset_index(drop=True) # Just the RGB images
Train.head()

,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_005sitfgr2.jpg,brown
2,id_00stp9t6m6.jpg,blast
3,id_012zxewnhx.jpg,blast
4,id_0186qwq2at.jpg,healthy


In [ ]:
targets = Train.Label.unique().tolist()

Target_Mapper = dict(zip(targets,[i for i in range(len(targets))]))
InverseTarget_Mapper = dict(zip([i for i in range(len(targets))],targets))

In [ ]:
Train.Label = Train.Label.map(Target_Mapper)

In [ ]:
Test = pd.read_csv('Test.csv')
Test = Test[~Test.Image_id.str.contains('_rgn')].reset_index(drop=True) # Just the RGB images

# Weighted Blend

## Utils

In [ ]:
test_dict = {
            'SwinLarge384': 'preds/AssazzinBaseline_swin_large_patch4_window12_384_Preds_gmean_10Folds.npy', 
            'SwinLarge256_V2': 'preds/AssazzinBaseline_swinv2_large_window12to16_192to256_22kft1k_Preds_gmean_10Folds.npy', 
            'SwinLarge384_V2': 'preds/AssazzinBaseline_swinv2_base_window12to24_192to384_22kft1k_gmean_10Folds.npy', 
           }

BlendTest = np.zeros((len(test_dict), Test.shape[0],y_true.shape[1]))
for i in range(BlendTest.shape[0]):
    BlendTest[i] = np.load(list(test_dict.values())[i])

In [ ]:
meanPreds = np.average(BlendTest , weights = np.array([0.34520169, 0.31686492, 0.33793339]), axis = 0) 

In [ ]:
sub = pd.DataFrame(meanPreds)
sub = sub.rename(columns=InverseTarget_Mapper)
sub['Image_id'] = Test['Image_id'].values

submission = pd.read_csv('SampleSubmission.csv')
sub = pd.merge(sub,submission[['Image_id']],on='Image_id',how='right')
for col in submission.columns[1:] :
  submission[col] = sub[col]

In [ ]:
submission.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999999,2.173010e-07,5.848037e-08
1,id_01hu05mtch.jpg,0.000044,9.992781e-01,1.434188e-06
2,id_030ln10ewn.jpg,0.805992,1.189005e-02,2.510919e-05
3,id_03z57m8xht.jpg,0.999999,9.121373e-08,3.935138e-08
4,id_04ngep1w4b.jpg,0.999948,3.914551e-06,5.811045e-06


In [ ]:
submission.to_csv(f'AssazzinSolution.csv',index=False)